In [1]:
import os
import json

directory_path='./responses'
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
print(files)

total_in_tokens = total_out_tokens = 0
in_price = 0.15 / 1_000_000
out_price = 0.6 / 1_000_000

for f in files:
    with open(f'{directory_path}/{f}', 'r', encoding='utf-8') as file:
        data=json.load(file)
        total_out_tokens += data['usage']['completion_tokens']
        total_in_tokens += data['usage']['prompt_tokens']

avg_out_tokens = total_out_tokens / len(files)
avg_in_tokens = total_in_tokens / len(files)

print(avg_in_tokens)
print(avg_out_tokens)
print(avg_in_tokens * 1_000_000 * in_price)
print(avg_out_tokens * 1_000_000 * out_price)

['0000020-15.2024.8.26.0595.json', '0008426-88.2023.8.26.0068.json', '1000720-80.2024.8.26.0010.json', '1002234-63.2022.8.26.0002.json', '1002770-47.2024.8.26.0344.json', '1003547-98.2023.8.26.0495.json', '1003793-28.2024.8.26.0344.json', '1008016-91.2024.8.26.0451.json', '1021317-42.2023.8.26.0451.json', '1028559-20.2018.8.26.0001.json']
303.6
74.9
45.54
44.94


In [27]:
def values_match(val1, val2):
    # Treat None and '' as a match
    return (val1 == val2) or (val1 in [0, None, ''] and val2 in [0, None, ''])

print(values_match(None, ""))
print(values_match(None, 0))

True
True


In [29]:
def compare_decisions(list1, list2, verbose):
    # Initialize counters
    total_requests = len(list1)
    matched_requests = 0
    matched_decision_types = 0
    matched_values = 0

    for item1 in list1:
        # Find the matching request in the second list
        matched_item = next((item2 for item2 in list2 if item2['pedido'] == item1['pedido']), None)

        if matched_item:
            matched_requests += 1
            
            # Compare decision type
            if values_match(item1.get('tipoDaDecisao'), matched_item.get('tipoDaDecisao')):
                matched_decision_types += 1
            
            # Compare granted values
            if values_match(item1.get('valorDeferido'), matched_item.get('valorDeferido')):
                matched_values += 1
            else: 
                print("value didnt match")
                print(item1.get('valorDeferido'))
                print(matched_item.get('valorDeferido'))
        
            # Print details of each comparison
            if verbose:
                print(f"Comparing '{item1['pedido']}':")
                print(f"  Decision Type: {item1.get('tipoDaDecisao')} vs {matched_item.get('tipoDaDecisao') if matched_item else 'N/A'}")
                print(f"  Granted Value: {item1.get('valorDeferido')} vs {matched_item.get('valorDeferido') if matched_item else 'N/A'}")
                print(f"  Obligation: {item1.get('obrigacaoDeFazer')} vs {matched_item.get('obrigacaoDeFazer') if matched_item else 'N/A'}\n")
        
        else:
            if verbose:
                print(f"Didn't find '{item1['pedido']}'! \n")
            

    # Calculate and print summary
    total_decision_types = total_requests
    total_values = sum(1 for item in list1 if 'valorDeferido' in item)

    if verbose:
        print(f"Todos os pedidos: {matched_requests}/{total_requests}")
        print(f"Classificação dos pedidos: {matched_requests}/{total_requests}")
        print(f"Classificação das decisões: {matched_decision_types}/{total_decision_types}")
        print(f"Valores: {matched_values}/{total_values}")

    return {
        "total_requests": total_requests,
        "matched_requests": matched_requests,
        "matched_decision_types": matched_decision_types,
        "matched_values": matched_values 
    }

# Example lists to compare
list1 = [{'pedido': 'Dano material', 'tipoDaDecisao': 'Procedente', 'valorDeferido': 2400, 'obrigacaoDeFazer': ''},
         {'pedido': 'Dano moral', 'tipoDaDecisao': 'Procedente', 'valorDeferido': 5000, 'obrigacaoDeFazer': ''}]

list2 = [{'pedido': 'Obrigação de fazer', 'tipoDaDecisao': 'Procedente', 'obrigacaoDeFazer': 'Estorno da compra no valor de R$ 2.400,00, realizada aos 28 de setembro de 2023, reconduzindo a relação à situação anterior à mencionada utilização.'},
         {'pedido': 'Dano moral', 'tipoDaDecisao': 'Procedente', 'valorDeferido': 5000}]

# Perform the comparison
compare_decisions(list1, list2, False)


{'total_requests': 2,
 'matched_requests': 1,
 'matched_decision_types': 1,
 'matched_values': 1}

In [30]:
with open('all_results.json', 'r', encoding='utf-8') as file:
    results = json.load(file)

with open('data.json', 'r', encoding='utf-8') as file:
    samples = json.load(file)

def sum_comparisons(comparison_results):
    # Initialize the accumulated totals
    totals = {
        'total_requests': 0,
        'matched_requests': 0,
        'matched_decision_types': 0,
        'matched_values': 0
    }

    # Sum the results from each comparison
    for result in comparison_results:
        for key in totals:
            totals[key] += result[key]

    print(totals)


comparison_results=[]
for result in results:
    id = result['id']
    item_with_id = [item for item in samples if item["id"] == id][0]
    comparison_results.append(compare_decisions(item_with_id['decisoes'], result['decisoes'], True))

sum_comparisons(comparison_results)

# checar o número de pedidos
# checar a classificação do pedido
# checar a classificação da decisão
# checar o valor deferido

Didn't find 'Obrigação de fazer'! 

Comparing 'Dano moral':
  Decision Type: Procedente vs Procedente
  Granted Value: 5000 vs 5000
  Obligation: None vs 

Todos os pedidos: 1/2
Classificação dos pedidos: 1/2
Classificação das decisões: 1/2
Valores: 1/1
Comparing 'Dano material':
  Decision Type: Procedente vs Procedente
  Granted Value: 77410.78 vs 77410.78
  Obligation: None vs 

Todos os pedidos: 1/1
Classificação dos pedidos: 1/1
Classificação das decisões: 1/1
Valores: 1/1
Comparing 'Declaratória':
  Decision Type: Parcialmente procedente vs Parcialmente procedente
  Granted Value: None vs 0
  Obligation: Rescisão do contrato de prestação de serviços firmado entre as partes, consubstanciado na reserva registrada sob o nº RES082702-5705, referente às diárias no Hotel Laghetto Stilo Barra Rio, por culpa exclusiva da requerida, sem qualquer ônus ao consumidor. vs Declarar a rescisão do contrato de prestação de serviços.

Comparing 'Dano material':
  Decision Type: Parcialmente proced